<a href="https://colab.research.google.com/github/ArushiG11/Anime-Data-Analysis/blob/main/Anime_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/manami-project/anime-offline-database?tab=readme-ov-file

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, re

aniSearch : https://www.anisearch.com/anime/toplist

In [2]:
animes = []
for pg in range(1,51):
  url_as = f'https://www.anisearch.com/anime/toplist/page-{pg}'
  header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
  }
  page_as = requests.get(url_as, headers=header)
  soup = BeautifulSoup(page_as.text,'html')
  ul = soup.find('ul',class_ = 'covers fullsizeA')
  items = ul.find_all('li')

  for item in items:
    anime = item.find('span', class_='details')
    type_num_date = anime.find('span', class_='date').text
    type_num_date = type_num_date.split()
    anime_type = type_num_date[0][:-1]
    num_episodes = type_num_date[1]
    year = type_num_date[2][1:-1]
    title = anime.find('span', class_='title').text
    company_span = anime.find('span', class_='company')
    company = company_span.text if company_span else None
    rank = item.find('div',class_="rank").text[1:]
    rank = int(rank)
    ratings = item.find('div',class_='star0').attrs['title']
    ratings = ratings.split()
    rating = ratings[0]
    comment = ratings[1]
    rating = rating.replace(',','')
    rating = float(rating)
    genre = item.find('div', class_='genre').text
    animes.append([rank, title, year, num_episodes, genre, anime_type, company, rating, comment])
df_as = pd.DataFrame(animes,columns=['Rank', 'Title', 'Year', 'Number of Episodes', 'Genre', 'Anime Type', 'Production House', 'Ratings', 'Comments'])

BoxOffice Mojo: https://www.boxofficemojo.com/genre/sg4259246337/, \
              https://www.boxofficemojo.com/genre/sg4259246337/?offset=100

In [4]:
urls_boxof = ["https://www.boxofficemojo.com/genre/sg4259246337/", \
              "https://www.boxofficemojo.com/genre/sg4259246337/?offset=100"]
page_boxof = []
for idx,i in enumerate(urls_boxof):
    r_boxof = requests.get(i)
    page_boxof.append(pd.read_html(r_boxof.text))
    print("Boxoffice page is scanned: " + str(idx+1) + "/" + str(len(urls_boxof)), end='\r')

<ipython-input-4-3e5491320460>:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  page_boxof.append(pd.read_html(r_boxof.text))


<ipython-input-4-3e5491320460>:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  page_boxof.append(pd.read_html(r_boxof.text))


In [5]:
# boxoffice data
# observe the shapes
print("Shapes", page_boxof[0][0].shape, page_boxof[1][0].shape)

# combine databases obtained for each page
df_boxof = pd.concat([page_boxof[0][0], page_boxof[1][0]], axis=0, ignore_index=True)

# observe the new dataframe details
df_boxof.info(verbose=True)

Shapes (100, 8) (26, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Rank            126 non-null    int64 
 1   Title           126 non-null    object
 2   Lifetime Gross  126 non-null    object
 3   Max Theaters    126 non-null    object
 4   Opening         126 non-null    object
 5   Open Th         126 non-null    object
 6   Release Date    126 non-null    object
 7   Distributor     126 non-null    object
dtypes: int64(1), object(7)
memory usage: 8.0+ KB


My Anime List : https://myanimelist.net/topanime.php

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def get_page_with_retries(session, url, max_retries=5, initial_delay=1):
    for i in range(max_retries):
        try:
            response = session.get(url)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            if i < max_retries - 1:
                delay = initial_delay * (2 ** i) + random.uniform(0, 1)
                print(f"Retrying in {delay:.2f} seconds...")
                time.sleep(delay)
    return None

def scrape_anime_data(max_limit=200):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

    anime_list = []
    for limit in range(0, max_limit + 1, 50):
        urls_mal = f'https://myanimelist.net/topanime.php?limit={limit}'
        page = get_page_with_retries(session, urls_mal)
        if not page:
            print(f"Failed to fetch page at limit {limit}")
            continue

        print(f"Limit: {limit}, Status Code: {page.status_code}")
        soup = BeautifulSoup(page.text, 'html.parser')
        table = soup.find('table', class_='top-ranking-table')
        if not table:
            print(f"No table found at limit {limit}")
            continue
        table_data = table.find_all('tr', class_='ranking-list')

        for row in table_data:
            try:
                rank = int(row.find('td', class_='rank ac').text.strip())
                title_cell = row.find('td', class_='title al va-t word-break')
                link_cell = title_cell.find('a')
                href = link_cell.get('href')
                if href:
                    detail_response = get_page_with_retries(session, href)
                    if not detail_response:
                        print(f"Failed to fetch details for anime at {href}")
                        continue
                    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

                    # Extract information (same as before)
                    Name_element = detail_soup.find('h1', class_='title-name h1_bold_none')
                    Name = Name_element.text.strip() if Name_element else 'N/A'
                    english_name_element = detail_soup.find('p', class_='title-english title-inherit')
                    English_Name = english_name_element.text.strip() if english_name_element else 'N/A'
                    Synopsis_element = detail_soup.find('p', itemprop='description')
                    Synopsis = Synopsis_element.text.strip() if Synopsis_element else 'N/A'
                    score_element = detail_soup.find('div', class_='fl-l score')
                    score = float(score_element.find('div').text.strip()) if score_element else 0.0
                    Information_element = detail_soup.find_all('div', class_='spaceit_pad')
                    Information = {
                        'Type': 'N/A',
                        'Episodes': 0,
                        'Status': 'N/A',
                        'Aired': 'N/A',
                        'Season': 'N/A',
                        'Studios': 'N/A',
                        'Producers': 'N/A',
                        'Genres': 'N/A',
                        'Demographics': 'N/A',
                        'Source': 'N/A',
                        'Popularity': 0,
                        'Members': 'N/A'
                    }

                    for element in Information_element:
                        text = element.text.strip()
                        if 'Type:' in text:
                            Information['Type'] = element.find('a').text.strip()
                        elif 'Episodes:' in text:
                            Information['Episodes'] = text.split(':')[-1].strip()
                        elif 'Status:' in text:
                            Information['Status'] = text.split(':')[-1].strip()
                        elif 'Aired:' in text:
                            Information['Aired'] = text.split(':')[-1].strip()
                        elif 'Premiered:' in text:
                            Information['Season'] = text.split(':')[-1].strip()
                        elif 'Studios:' in text:
                            Information['Studios'] = element.find('a').text.strip()
                        elif 'Producers:' in text:
                            Information['Producers'] = element.find('a').text.strip()
                        elif 'Genres:' in text:
                            Information['Genres'] = element.find('a').text.strip()
                        elif 'Demographic:' in text:
                            demographics_span = element.find('span', itemprop='genre')
                            Information['Demographics'] = demographics_span.text.strip() if demographics_span else 'N/A'
                        elif 'Source:' in text:
                            Information['Source'] = text.split(':')[-1].strip()
                        elif 'Popularity:' in text:
                            Information['Popularity'] = int(text.split(':')[-1].strip().replace('#', ''))
                        elif 'Members:' in text:
                            Information['Members'] = text.split(':')[-1].strip()

                    anime_list.append([rank, Name, English_Name, Synopsis, score, Information['Popularity'], Information['Type'],
                                      Information['Episodes'], Information['Status'], Information['Aired'], Information['Season'],
                                      Information['Studios'], Information['Producers'], Information['Genres'], Information['Demographics'],
                                      Information['Members'], Information['Source']])

                    # Add a small delay between requests to individual anime pages
                    time.sleep(random.uniform(1, 3))
            except Exception as e:
                print(f"Error processing row: {e}")

        # Add a longer delay between main page requests
        delay = random.uniform(5, 10)
        print(f"Sleeping for {delay:.2f} seconds...")
        time.sleep(delay)

    df_mal = pd.DataFrame(anime_list, columns=['Rank', 'Name', 'English Name', 'Synopsis', 'Score', 'Popularity', 'Type',
                                                            'Number of Episodes', 'Status', 'Aired', 'Season', 'Studios',
                                                            'Producers', 'Genres', 'Demographics', 'Members', 'Source'])

    return df_mal

df_mal = scrape_anime_data(max_limit=200)

Limit: 0, Status Code: 200
Sleeping for 8.25 seconds...
Limit: 50, Status Code: 200
Sleeping for 6.95 seconds...
Limit: 100, Status Code: 200
Sleeping for 7.92 seconds...
Limit: 150, Status Code: 200
Sleeping for 7.73 seconds...
Limit: 200, Status Code: 200
Sleeping for 8.50 seconds...


In [7]:
# save raw data for emergency uses
if not os.path.exists('data'):
    os.mkdir('data')
df_boxof.to_csv(os.path.join('data', 'raw_boxof.csv'), encoding='utf-8-sig')
df_as.to_csv(os.path.join('data','aniSearch.csv'), encoding='utf-8-sig')
df_mal.to_csv(os.path.join('data','MyAnimeList.csv'), encoding='utf-8-sig')

Build a Comprehensive Dataset

In [8]:
df_mal

,Rank,Name,English Name,Synopsis,Score,Popularity,Type,Number of Episodes,Status,Aired,Season,Studios,Producers,Genres,Demographics,Members,Source
0,1,Sousou no Frieren,Frieren: Beyond Journey's End,During their decade-long quest to defeat the D...,9.32,191,TV,28,Finished Airing,"Sep 29, 2023 to Mar 22, 2024",Fall 2023,Madhouse,Aniplex,Adventure,Shounen,"929,728",Manga
1,2,One Piece Fan Letter,N/A,Two years after the Summit War in which Straw ...,9.22,3547,TV Special,1,Finished Airing,"Oct 20, 2024",N/A,Toei Animation,add some,Action,Shounen,"42,696",Light novel
2,3,Fullmetal Alchemist: Brotherhood,N/A,After a horrific alchemy experiment goes wrong...,9.09,3,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Bones,Aniplex,Action,Shounen,"3,431,405",Manga
3,4,Steins;Gate,N/A,Eccentric scientist Rintarou Okabe has a never...,9.07,14,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,White Fox,Frontier Works,Drama,N/A,"2,628,828",Visual novel
4,5,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,Seeking to restore humanity's diminishing hope...,9.05,21,TV,10,Finished Airing,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,Wit Studio,Production I.G,Action,Shounen,"2,359,290",Manga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,246,Natsume Yuujinchou: Itsuka Yuki no Hi ni,Natsume‚Äôs Book of Friends: On a Certain Snow...,As the first delicate snowflake touches the co...,8.33,2458,OVA,1,Finished Airing,"Feb 5, 2014",N/A,Brain's Base,Aniplex,Slice of Life,Shoujo,"81,958",Manga
246,247,One Outs,N/A,Toua Tokuchi is a prodigy when it comes to bot...,8.33,1035,TV,25,Finished Airing,"Oct 8, 2008 to Apr 1, 2009",Fall 2008,Madhouse,VAP,Sports,Seinen,"244,172",Manga
247,248,Psycho-Pass,N/A,"Justice, and the enforcement of it, has change...",8.33,62,TV,22,Finished Airing,"Oct 12, 2012 to Mar 22, 2013",Fall 2012,Production I.G,Dentsu,Action,N/A,"1,666,700",Original
248,249,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,Re:ZERO -Starting Life in Another World- Season 2,A reunion that was supposed to spell the arriv...,8.33,130,TV,13,Finished Airing,"Jul 8, 2020 to Sep 30, 2020",Summer 2020,White Fox,AT-X,Drama,N/A,"1,151,520",Light novel


In [9]:
df_mal.dtypes

,0
Rank,int64
Name,object
English Name,object
Synopsis,object
Score,float64
Popularity,int64
Type,object
Number of Episodes,object
Status,object
Aired,object


In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
def clean_mal_data(df_mal):
    """Clean and preprocess MyAnimeList data"""
    df_mal_clean = df_mal.copy()

    # Clean episode numbers
    df_mal_clean['Number of Episodes'] = df_mal_clean['Number of Episodes'].replace('Unknown', '0')
    df_mal_clean['Number of Episodes'] = pd.to_numeric(df_mal_clean['Number of Episodes'], errors='coerce')

    # Extract year from Aired column
    def extract_year(aired_str):
        try:
            if pd.isna(aired_str) or aired_str == 'N/A':
                return np.nan
            # Extract first year mentioned
            year = aired_str.split(',')[0].split(' ')[-1]
            return int(year) if year.isdigit() else np.nan
        except:
            return np.nan

    df_mal_clean['Year'] = df_mal_clean['Aired'].apply(extract_year)

    # Clean Members column
    df_mal_clean['Members'] = df_mal_clean['Members'].str.replace(',', '').astype(float)

    return df_mal_clean

In [13]:
df_mal_clean = clean_mal_data(df_mal)

In [15]:
df_mal_clean.head()

,Rank,Name,English Name,Synopsis,Score,Popularity,Type,Number of Episodes,Status,Aired,Season,Studios,Producers,Genres,Demographics,Members,Source,Year
0,1,Sousou no Frieren,Frieren: Beyond Journey's End,During their decade-long quest to defeat the D...,9.32,191,TV,28,Finished Airing,"Sep 29, 2023 to Mar 22, 2024",Fall 2023,Madhouse,Aniplex,Adventure,Shounen,929728.0,Manga,29
1,2,One Piece Fan Letter,N/A,Two years after the Summit War in which Straw ...,9.22,3547,TV Special,1,Finished Airing,"Oct 20, 2024",N/A,Toei Animation,add some,Action,Shounen,42696.0,Light novel,20
2,3,Fullmetal Alchemist: Brotherhood,N/A,After a horrific alchemy experiment goes wrong...,9.09,3,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Bones,Aniplex,Action,Shounen,3431405.0,Manga,5
3,4,Steins;Gate,N/A,Eccentric scientist Rintarou Okabe has a never...,9.07,14,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,White Fox,Frontier Works,Drama,N/A,2628828.0,Visual novel,6
4,5,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,Seeking to restore humanity's diminishing hope...,9.05,21,TV,10,Finished Airing,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,Wit Studio,Production I.G,Action,Shounen,2359290.0,Manga,29


In [18]:
df_mal_clean.dtypes

,0
Rank,int64
Name,object
English Name,object
Synopsis,object
Score,float64
Popularity,int64
Type,object
Number of Episodes,int64
Status,object
Aired,object
